# `p2oc` CLI Demo

Open a double-funded channel without the aid of an intermediary being present.

In [ ]:
%load_ext autoreload
%autoreload 2

## Fund Ali and Bob

In [ ]:
import sys
sys.path.append('/home/jovyan/lnrpc')
sys.path.append('/notebooks/p2oc/p2oc')

import os
import time
import bitcoin
from btc_rpc import Proxy, Config
from lnd_rpc import LndRpc, lnmsg

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
ali = LndRpc(host='ali-lnd:10009', cert_path='/ali-lnd/tls.cert', macaroon_path='/tmp/ali.macaroon')
bob = LndRpc(host='bob-lnd:10009', cert_path='/bob-lnd/tls.cert', macaroon_path='/tmp/bob.macaroon')

bitcoin.SelectParams('regtest')

brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect=os.environ['BTCD_RPCHOST'],
    rpcport=os.environ['BTCD_RPCPORT']
))

In [ ]:
ali_address = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))
bob_address = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))

try:
    brpc.loadwallet('miner')
except:
    brpc.createwallet('miner')
    
miner_address = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

In [ ]:
_ = list(brpc.generatetoaddress(400, miner_address))
_ = list(brpc.generatetoaddress(10, ali_address.address))
_ = list(brpc.generatetoaddress(10, bob_address.address))
_ = list(brpc.generatetoaddress(120, miner_address))

# Give LNDs time to catch up
time.sleep(5)

In [ ]:
print(f"""alice balance={ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())}
bob balance={bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())}""")

## Create Double-Funded Channel

In [ ]:
!pip install -e ./p2oc
!p2oc --help

### Alice (Taker)

Creates public offer to request inbound liquidity in exchange for a fee.

```bash
# from host
docker exec -it jupyter bash

LND_ENDPOINT="ali-lnd:10009" \
LND_CERTPATH="/ali-lnd/tls.cert" \
LND_MRNPATH="/tmp/ali.macaroon" \
    p2oc createoffer --request_fund_amount=16000000 --premium_amount=100000
```

### Bob (Maker)

Accepts offer and sends funding transaction response.

```bash
# from host
docker exec -it jupyter bash

LND_ENDPOINT="bob-lnd:10009" \
LND_CERTPATH="/bob-lnd/tls.cert" \
LND_MRNPATH="/tmp/bob.macaroon" \
    p2oc createchannelfromoffer <offer>
```

## Confirm Channel Opening

In [ ]:
_ = list(brpc.generatetoaddress(6, miner_address))

In [ ]:
# should be no pending channels
assert len(ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest()).pending_open_channels) == 0

ali.lnd.ListChannels(lnmsg.ListChannelsRequest())